In [1]:
# Generate data to infer the rules for integral

import json
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot, evalf, N
from IPython.display import display
from utils.utils import *


def remove_constants(f):
    t = Symbol('t')
    return f.as_independent(t)[1]

fin = open("datasets/parametric_equations_polynomial_integral_results.json", "r")
lines = fin.readlines()
fin.close()
fin = open("datasets/parametric_equations_randomized_polynomial_integral_results.json", "r")
lines.extend(fin.readlines())
fin.close()

MAX_POWER = 6
MAX_AVG_DIFF = 0.01

originals = []
integrals = []
t = Symbol('t')

for line in lines:
    result = json.loads(line)
    if "rounded_regressed" not in result:
        continue
    original = round_all_floats(N(sympify(result["original"])))
    integral = remove_constants(round_all_floats(N(sympify(result["rounded_regressed"]))))
    #try:
    original = filter_non_polynomial(original)
    integral = filter_non_polynomial(integral)
    original_integral = integrate(original, t)
    avg_diff = get_avg_diff(original_integral, integral, t)
    if avg_diff > MAX_AVG_DIFF or len(original_integral.args) != len(integral.args):
        print("Skipping. Diff=", avg_diff)
        display(original_integral)
        display(integral)
        continue
#     except:
#         print("Cannot filter non-polynomials on", str(integral))
#         continue
    originals.append(str(original))
    integrals.append(str(integral))
    if len(originals) % 100 == 0:
        print(len(originals), "cases loaded")
    
fin.close()


Skipping. Diff= 1.3225806451612898


-3.1*t**2

t**2

Skipping. Diff= 0.6699999999999996


0.33*t**2

...

11.266*t**5 + 56.335*t**2 + 56.33*t

11.27*t**5 - 0.02*t**4 - 0.09*t**3 + 56.06*t**2 + 55.94*t

Skipping. Diff= 0.013532121282198495


0.066016*t**5 + 8.665*t**2 + 225.33*t

1.37*t**3 + 8.16*t**2 + 220.02*t

In [2]:
f = sp.sympify("1.2*t**5 + 0.98*t**4 + 0.32*t**3 + 0.52*t**2 + 2.81*t - 0.09")
t = Symbol('t')
display(f.as_independent(t)[1])

1.2*t**5 + 0.98*t**4 + 0.32*t**3 + 0.52*t**2 + 2.81*t

In [2]:
import json
import torch
import random
from datasets import load_dataset, Dataset, load_from_disk

for i in range(len(originals)):
    originals[i] = originals[i] + ' repeat ' + originals[i]

ds = Dataset.from_dict({'question': originals, 'answer':integrals})
ds = ds.shuffle()
train_ds = ds.train_test_split(test_size=0.05)

train_ds['train'][1]

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'question': '9.0*t - 15.0 repeat 9.0*t - 15.0', 'answer': '4.5*t**2 - 15.0*t'}

In [3]:
train_ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 18108
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 954
    })
})

In [4]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-large"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

CONTEXT_LENGTH = 128

# We prefix our tasks with "answer the question"
prefix = ""

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=CONTEXT_LENGTH, truncation=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=CONTEXT_LENGTH,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

tokenized_dataset = train_ds.map(preprocess_function, batched=True)

# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m") #"EleutherAI/gpt-neo-125m") "xhyi/PT_GPTNEO350_ATG"
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

# def preprocess_function(examples):
#     return tokenizer(examples["eq_pair"], padding='max_length', truncation=True, max_length=CONTEXT_LENGTH, return_tensors="pt")

# tokenized_ds = train_ds.map(
#     preprocess_function,
#     batched=True,
#     num_proc=1,
#     remove_columns=train_ds["train"].column_names,
# )

# def preprocess_function(examples):
#     examples["labels"] = examples["input_ids"].copy()
#     return examples

# lm_dataset = tokenized_ds.map(preprocess_function, batched=True, num_proc=1)

# from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 954/954 [00:00<00:00, 16790.10 examples/s]


In [5]:
tokenized_dataset['test'][1]

{'question': '49.0*t**3 + 79.33*t + 32.11 repeat 49.0*t**3 + 79.33*t + 32.11',
 'answer': '12.25*t**4 + 39.67*t**2 + 32.11*t',
 'input_ids': [9526,
  5,
  632,
  1935,
  17,
  19844,
  519,
  1768,
  3,
  4440,
  5,
  4201,
  1935,
  17,
  1768,
  3538,
  5,
  2596,
  6103,
  9526,
  5,
  632,
  1935,
  17,
  19844,
  519,
  1768,
  3,
  4440,
  5,
  4201,
  1935,
  17,
  1768,
  3538,
  5,
  2596,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [8013,
  1828,
  1935,
  17,
  19844,
  591,
  1768,
  6352,
  5,
  3708,
  1935,
  17,
  19844,
  357,
  1768,
  3538,
  5,
  2596,
  1935,
  17,
  1]}

In [6]:
train_ds = tokenized_dataset['train']

for i in range(1000, min(len(train_ds), 1030)):
    question = train_ds[i]['question'] 
    if len(question) > 3:
        print("Case", i)
        print(question)
        print(tokenizer.decode(train_ds[i]['labels']))

Case 1000
33.33*t**2 + 33.33*t + 8.33 repeat 33.33*t**2 + 33.33*t + 8.33
11.11*t**3 + 16.67*t**2 + 8.33*t</s>
Case 1001
4.67*t + 6.67 repeat 4.67*t + 6.67
2.33*t**2 + 6.67*t</s>
Case 1002
46.0*t**3 + 208.0*t + 225.33 repeat 46.0*t**3 + 208.0*t + 225.33
11.5*t**4 + 104.0*t**2 + 225.33*t</s>
Case 1003
66.0*t**3 - 16.0*t + 1.0 repeat 66.0*t**3 - 16.0*t + 1.0
16.5*t**4 - 8.0*t**2 + 1.0*t</s>
Case 1004
-0.67*t - 4.0 repeat -0.67*t - 4.0
-0.33*t**2 - 4.0*t</s>
Case 1005
7.0*t**3 - 3.0*t + 16.0 repeat 7.0*t**3 - 3.0*t + 16.0
1.75*t**4 - 1.5*t**2 + 16.0*t</s>
Case 1006
-3.0*t - 2.93 repeat -3.0*t - 2.93
-1.5*t**2 - 2.93*t</s>
Case 1007
3.29*t - 6.14 repeat 3.29*t - 6.14
1.64*t**2 - 6.14*t</s>
Case 1008
5.0*t**2 - 8.65*t + 3.27 repeat 5.0*t**2 - 8.65*t + 3.27
1.67*t**3 - 4.33*t**2 + 3.27*t</s>
Case 1009
2.75*t + 4.25 repeat 2.75*t + 4.25
1.38*t**2 + 4.25*t</s>
Case 1010
15.71*t**3 + 1.83*t**2 + 14.06*t + 5.08 repeat 15.71*t**3 + 1.83*t**2 + 14.06*t + 5.08
3.93*t**4 + 0.61*t**3 + 7.03*t**2 + 5.0

In [7]:
import nltk
import evaluate
import numpy as np

#nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds
   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
   return result

# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 2
NUM_EPOCHS = 5
SAVE_STEPS=1000

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="datasets/integrate_flant5_20240101",
   evaluation_strategy="steps",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   save_steps=SAVE_STEPS,
   eval_steps=SAVE_STEPS,
   logging_steps=SAVE_STEPS,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

#cp_path = "datasets/integrate_flant5_20240101/checkpoint-36000"

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1000,0.256000,0.129291,0.930856,0.883982,0.929912,0.929957
2000,0.116400,0.093237,0.949084,0.914346,0.948694,0.948612
3000,0.087400,0.084233,0.951720,0.919723,0.951342,0.951139
4000,0.075300,0.072178,0.955850,0.927423,0.955727,0.955614
5000,0.059900,0.067055,0.958283,0.931620,0.957812,0.957688
6000,0.053200,0.063085,0.958526,0.932788,0.958511,0.958489
7000,0.048500,0.060410,0.960572,0.936223,0.960353,0.960447
8000,0.036100,0.059241,0.962001,0.939351,0.961911,0.961800
9000,0.034500,0.059523,0.960710,0.936224,0.960640,0.960504
10000,0.024700,0.061351,0.962964,0.940986,0.962983,0.962832


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transforme

TrainOutput(global_step=11320, training_loss=0.07281472885145315, metrics={'train_runtime': 3180.7201, 'train_samples_per_second': 28.465, 'train_steps_per_second': 3.559, 'total_flos': 1.663808382517248e+16, 'train_loss': 0.07281472885145315, 'epoch': 5.0})

In [14]:
model.save_pretrained("datasets/integrate_flant5_20240101/flant5-large-11000-loss0.050.model")

In [7]:
#model = AutoModelForCausalLM.from_pretrained("datasets/normalize_symbolic_regression_results_20231219/gptneo-350m-22000-loss0.443.model")

In [19]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


#inputs = tokenizer(inputs, return_tensors="pt").to(device)

def generate_integral(inputs):
    inputs = tokenizer(text_target=inputs, 
                       max_length=CONTEXT_LENGTH,         
                       truncation=True,
                       return_tensors="pt").to(device)
    #print(inputs)
    outputs = model.generate(**inputs, temperature=0.01)
    answer = tokenizer.decode(outputs[0])
    answer = answer.replace('<pad>','').replace('</s>','').strip()
    if answer[-1]=='*':
       answer = answer+'t'
    return answer

print(generate_integral("2.5*t**3 + 0.51*t**2 + 68.55"))

# # Encode some input text
# prompt = "0.33*t**3 - 1.0*t**2 entail"
# input_ids = tokenizer.encode(prompt, return_tensors='pt')

# # Generate text
# output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.7)

# # Decode and print the output
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(generated_text)

0.63*t**4 + 0.17*t**3 + 68.55*t


In [15]:
tokenized_dataset['test'][0]

{'question': '78.45*t**3 + 70.86*t + 14.0',
 'answer': '19.61*t**4 + 35.43*t**2 + 14.0*t',
 'input_ids': [3,
  3940,
  5,
  2128,
  1935,
  17,
  19844,
  519,
  1768,
  2861,
  5,
  3840,
  1935,
  17,
  1768,
  209,
  15021,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [9997,
  4241,
  1935,
  17,
  19844,
  591,
  1768,
  3097,
  5,
  4906,
  1935,
  17,
  19844,
  357,
  1768,
  209,
  15021,
  1935,
  17,
  1]}

In [20]:
import sympy as sp
from utils.utils import *

MIN_ALLOWED_DIFF = 0.011

test_ds = tokenized_dataset['test']

num_processed = 0
num_equal = 0
num_zero_diff = 0
num_within_allowed = 0
t = sp.Symbol('t')

verbose = False

for i in range(min(len(test_ds), 1000)):
    if verbose:
        print("Case", i, test_ds[i]['question'])
    question = test_ds[i]['question']
    if 'repeat' in question:
        question = question[0:question.find('repeat')]
    original = sp.sympify(question)
    integral = round_all_floats(sp.integrate(original), 2)
    if verbose: display(integral)
    try:
        pred = generate_integral(test_ds[i]['question'])
        generated = round_all_floats(sp.sympify(pred), 2)
        if verbose: display(generated)
    except:
        print("Cannot sympify", pred)
        continue
    avg_diff = get_avg_diff(integral, generated, t)
    if verbose: print("avg_diff", avg_diff)
    num_processed += 1
    if avg_diff <= MIN_ALLOWED_DIFF:
        num_within_allowed += 1
    if avg_diff <= 0.0000001:
        num_zero_diff += 1
    diff_expr = sp.simplify(integral-generated)
    if verbose: display(diff_expr)
    if diff_expr == 0:
        if verbose: print("Equal")
        num_equal += 1
        
print("num_processed", num_processed)
print("num_equal", num_equal)
print("num_zero_diff", num_zero_diff)
print("num_within_allowed", num_within_allowed)

<lambdifygenerated-44890>:2: RuntimeWarning: invalid value encountered in power
  return 0.25*t**4 + 1.74*t**3 + 0.68*t**t
<lambdifygenerated-44974>:2: RuntimeWarning: invalid value encountered in power
  return 2.76*t**4 + 0.3*t**3 + 1.56*t**t
<lambdifygenerated-45024>:2: RuntimeWarning: invalid value encountered in power
  return 0.6*t**5 + 0.75*t**4 + 5.61*t**t
<lambdifygenerated-45058>:2: RuntimeWarning: invalid value encountered in power
  return 0.8*t**5 + 0.25*t**4 + 0.36*t**t


Cannot sympify 21.78*t**3 - 121.33*t**2 +


<lambdifygenerated-45120>:2: RuntimeWarning: invalid value encountered in power
  return 0.2*t**5 + 1.19*t**3 + 2.27*t**t
<lambdifygenerated-45124>:2: RuntimeWarning: invalid value encountered in power
  return 1.0*t**4 + 1.08*t**3 + 4.78*t**t
<lambdifygenerated-45210>:2: RuntimeWarning: invalid value encountered in power
  return 1.6*t**5 + 0.5*t**4 + 0.8*t**t
<lambdifygenerated-45240>:2: RuntimeWarning: invalid value encountered in power
  return 0.5*t**4 + 2.93*t**3 + 2.35*t**t
<lambdifygenerated-45246>:2: RuntimeWarning: invalid value encountered in power
  return 0.25*t**4 + 1.77*t**3 + 1.55*t**t
<lambdifygenerated-45646>:2: RuntimeWarning: invalid value encountered in power
  return 1.0*t**4 + 1.39*t**3 + 0.61*t**t
<lambdifygenerated-45704>:2: RuntimeWarning: invalid value encountered in power
  return 0.4*t**5 + 1.0*t**4 + 1.0*t**t
<lambdifygenerated-45708>:2: RuntimeWarning: invalid value encountered in power
  return 0.92*t**6 + 0.92*t**4 + 4.81*t**t
<lambdifygenerated-45720>:

num_processed 953
num_equal 645
num_zero_diff 645
num_within_allowed 813
